# Multimodal Financial Research Agent with Hybrid Search
Build an intelligent agent that can search financial documents (RAG) and live market data to provide comprehensive insights.

## Overview

This notebook covers:
- **Hybrid Search (RAG)**: Semantic + keyword search with automatic filter extraction from historical SEC filings
- **Live Finance Research**: Real-time stock data and market information via Yahoo Finance MCP
- **Agent with Dual Tools**: Connect LLM agent with both vector database and live APIs
- **Financial Analysis**: Query historical SEC filings (10-K, 10-Q) and current market data
- **Metadata Filtering**: Automatic extraction of company, year, quarter from queries

## Setup

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.agents import create_agent

from langchain.messages import HumanMessage, ToolMessage, AIMessage, SystemMessage
from langchain_core.tools import tool

from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3


In [4]:
from scripts.rag_tools import hybrid_search
from scripts.prompts import MULTIMODAL_AGENT_PROMPT

In [6]:
# Initialize Model
model = ChatGoogleGenerativeAI(model='gemini-3-flash-preview')

In [7]:
import subprocess
import sys

@tool
def live_finance_researcher(query: str):
    """
    Research live stock data using Yahoo Finance MCP.
    
    Use this tool to get:
    - Current stock prices and real-time market data
    - Latest financial news
    - Stock recommendations and analyst ratings
    - Option chains and expiration dates
    - Recent stock actions (splits, dividends)
    
    Args:
        query: The financial research question about current market data
        
    Returns:
        Research results from Yahoo Finance
    """

    code = f"""
import asyncio
from scripts.yahoo_mcp import finance_research
asyncio.run(finance_research("{query}"))
"""
    result = subprocess.run([sys.executable, '-c', code], capture_output=True, text=True)

    return result.stdout

## Create Financial Researcher Agent with Memory